In [1]:
import transformers

In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Create an engine to connect to the SQLite database
engine = create_engine('sqlite:////content/drive/MyDrive/ColabNotebooks/naver_news.db')

# Define the query to select the content column from the article_contents table
query = "SELECT content FROM article_contents"

# Use pandas to read the data from the database
df = pd.read_sql(query, engine)

# Display the DataFrame
print(df.head())

                                             content
0  \n피에스케이홀딩스는 피에스케이 주식회사가 2019년 인적 분할하면서 설립된 회사다...
1  \n-이번에는 금융권 소식을 들어보겠습니다. 미국 중앙은행인 연방준비제도(Fed)가...
2  \n\t\t\t[서울경제] <유가증권>▲보통주추가상장=(주식매수선택권행사) (국내C...
3  \n(서울=뉴스1) 박현영 기자 = 한국 진출을 예고했던 해외 대형 가상자산(암호화...
4  \n\t\t\t[이데일리 이정현 기자]AP시스템(265520)은 연결재무제표 기준 ...


# 1. 텍스트 분류 (감정 분석)
이제 한국어 뉴스 기사의 감정 분석을 위한 예제 코드를 작성합니다:

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# 감정 분석 모델과 토크나이저 로드
model_name = "kykim/bert-kor-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 파이프라인을 사용하여 감정 분석 수행
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# 샘플 데이터에 감정 분석 적용
sample_texts = df['content'].head(10).tolist()
results = classifier([text[:512] for text in sample_texts])

# 결과 출력
for text, result in zip(sample_texts, results):
    print(f"Text: {text[:100]}...")  # 긴 텍스트는 잘라서 출력
    print(f"Sentiment: {result['label']}, Score: {result['score']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text: 
피에스케이홀딩스는 피에스케이 주식회사가 2019년 인적 분할하면서 설립된 회사다. 전공정 장비는 피에스케이, 후공정 장비는 피에스케이홀딩스로 사업 분야가 나뉘었고 디스컴(Desc...
Sentiment: LABEL_1, Score: 0.5108184218406677
Text: 
-이번에는 금융권 소식을 들어보겠습니다. 미국 중앙은행인 연방준비제도(Fed)가 5월 연방공개시장위원회(FOMC)에서 6회 연속 기준금리를 동결하면서 한국은행의 셈법도 복잡해지고...
Sentiment: LABEL_0, Score: 0.5425594449043274
Text: 
			[서울경제] <유가증권>▲보통주추가상장=(주식매수선택권행사) (국내CB·BW) ▲주권변경상장=(주식소각)<코스닥>▲공모주청약=신한제13호스팩(주관사 신한) ▲보통주추가상장=...
Sentiment: LABEL_0, Score: 0.6045055985450745
Text: 
(서울=뉴스1) 박현영 기자 = 한국 진출을 예고했던 해외 대형 가상자산(암호화폐) 거래소 크립토닷컴이 오는 29일로 예정됐던 국내 서비스 출시를 무기한 연기했다. 금융당국이 제...
Sentiment: LABEL_1, Score: 0.5773926973342896
Text: 
			[이데일리 이정현 기자]AP시스템(265520)은 연결재무제표 기준 1분기 영업익이 전년동기대비 38.17% 감소한 72억3800만원이라고 8일 공시했다. 매출액은 2.33...
Sentiment: LABEL_1, Score: 0.5523409247398376
Text: 
한국디스플레이산업협회가 대만과의 마이크로 발광다이오드(LED) 기술교류를 강화한다.디스플레이산업협회는 23일부터 24일까지 이틀간 국내 소재·부품·장비(소부장) 기업들과 대만을 ...
Sentiment: LABEL_0, Score: 0.5136205554008484
Text: 
  지난달 28일 우주발사체 스타트업 '이노스페이스'가 코스닥 상장 예비 심사 승인 소식을 알렸다. 국

# 2. 텍스트 요약

다음으로, 뉴스 기사에 대한 텍스트 요약을 수행합니다:

In [7]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# 요약 모델과 토크나이저 로드
model_name = "tuner007/pegasus_paraphrase"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# 파이프라인을 사용하여 텍스트 요약 수행
summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)

# 샘플 데이터에 요약 적용
sample_texts = df['content'].head(5).tolist()
summaries = summarizer(sample_texts[0][:60], max_length=60, do_sample=False)

# 요약 결과 출력
for text, summary in zip(sample_texts, summaries):
    print(f"Original: {text[:]}...")  # 긴 텍스트는 잘라서 출력
    print(f"Summary: {summary['summary_text']}")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Your max_length is set to 60, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


Original: 
피에스케이홀딩스는 피에스케이 주식회사가 2019년 인적 분할하면서 설립된 회사다. 전공정 장비는 피에스케이, 후공정 장비는 피에스케이홀딩스로 사업 분야가 나뉘었고 디스컴(Descum) 장비, 리플로 장비, HDW 장비 등을 전문적으로 제조한다. 삼성전자, SK하이닉스, 마이크론 테크놀로지의 CAPEX는 대부분 고대역폭메모리(HBM) 시장에 집중된 상태다. 2024년 말 기준 HBM CAPA는 현재보다 최소 2배 이상 확대할 것으로 예상된다. HBM 제조 공정에서 가장 중요한 과정 중 하나가 TSV 공정이다. 향후 디스컴 장비 비중이 꾸준하게 증가할 것으로 예상되고 현재 피에스케이홀딩스 매출의 30% 수준까지 성장했다. 플립칩(FC) 본딩 증가로 평탄화용 리플로 장비 수요도 늘어나는 추세다. 삼성전자가 내년 3D D램을 공개한다고 선언한 만큼 메모리 반도체 적층 증가로 인한 수혜도 예상되는 상황이다. 비에이치의 사업 분야는 첨단 정보기술(IT) 산업의 핵심부품인 연성회로기판(FPCB) 사업과 차량용 휴대폰 무선충전기 개발 및 생산을 주력으로 하는 전장사업부로 분류된다. FPCB 사업은 첨단기술이 발전함에 따라 고기능·고집적 기술화된 빌드업과 RF 타입의 FPCB 수요가 증가하면서 당사의 기술력과 수익성이 함께 성장하고 있다. 비에이치의 1분기 매출액은 2988억원(전년 동기 대비 -4.8%), 영업이익은 84억원(전년 동기 대비 -5.5%)으로 영업이익 컨센서스(83억원)에 부합했다. 1분기 실적에는 다소 아쉬움이 있지만 계절적 성수기인 하반기가 다가오는 가운데 전장사업부의 수익성 기반이 확실해지고 있다는 점은 고무적이다. 한 증권사에서는 2분기 매출액을 3909억원, 영업이익을 172억원으로 전망했고, 북미 고객사향 경연성 회로기판 매출액이 예년보다 이른 시점부터 증가하기 시작할 것으로 예상된다.AP위성은 저궤도 위성통신 분야에서 빠르게 성장하고 있는 기업으로, 6G 기술의 발전에 큰 수혜를 볼 것으로 예상된다. AP위성은 인공위성 및 관련

# 3. 질문 답변
마지막으로, 뉴스 기사에 대한 질문 답변 태스크를 수행합니다:

In [8]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# 질문 답변 모델과 토크나이저 로드
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# 파이프라인을 사용하여 질문 답변 수행
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)

# 샘플 질문과 기사
sample_context = df['content'].iloc[3]
question = "이 기사에서 주요 인물은 누구인가요?"

# 질문 답변 수행
result = qa_pipeline(question=question, context=sample_context)

# 결과 출력
print(f"Context: {sample_context[:100]}...")  # 긴 텍스트는 잘라서 출력
print(f"Question: {question}")
print(f"Answer: {result['answer']}")

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Context: 
(서울=뉴스1) 박현영 기자 = 한국 진출을 예고했던 해외 대형 가상자산(암호화폐) 거래소 크립토닷컴이 오는 29일로 예정됐던 국내 서비스 출시를 무기한 연기했다. 금융당국이 제...
Question: 이 기사에서 주요 인물은 누구인가요?
Answer: 했다
